In [ ]:
# Имя набор данных построить аппроксимационную модель
# Хьюбер, Тюки - функция потерь - это робастость
# Квадрат если небольшое
# Линия если больше
# Константа
# 2 - регуляризация L1, L2, SVD
# 3 - робастость

In [ ]:
# 4 Лаб работа - сплайны. Натуральный сплайн
# 4 Лаб работа - сплайны. Робастный сглаживающий сплайн

In [ ]:
!python -m pip install matplotlib numpy scipy

In [ ]:
from sys import float_info

import numpy as np
from numpy.linalg import solve

def ordinary_least_squares(x: np.ndarray, y: np.ndarray, order: int,
        p: int = 0, alpha: float = 1.0, eps: float = 1E-8, max_iter: int = 100):
    if (not isinstance(x, np.ndarray) or not isinstance(y, np.ndarray)
            or not isinstance(order, int) or not isinstance(p, int)
            or not isinstance(alpha, float) or not isinstance(eps, float)
            or not isinstance(max_iter, int)):
        raise TypeError()
    if (len(x.shape) != 1 or len(y.shape) != 1 or x.shape != y.shape
            or order < 2 or p not in range(3) or alpha < 0
            or eps < float_info.epsilon or max_iter < 1):
        raise ValueError()

    n = len(x)
    x_upper = np.vander(x, order + 1, increasing=True).T
    left, right = x_upper.T @ x_upper, x_upper.T @ y
    print('Condition number =', np.linalg.cond(left))
    for _ in range(max_iter):
        next_a = solve(left, right)
    return poly

In [ ]:
from math import cos, sin

def f(x: float) -> float:
    return x + 10 * sin(x / 10) + 15 * sin(x / 5) + 20 * sin(x / 2)

x = np.linspace(-25, 25, 2 ** 20)
f_vectorize = np.vectorize(f)
y = f_vectorize(x)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16,9), dpi=400)
subplot = fig.add_subplot(111, facecolor = '#FFFFFF')
subplot.plot(x, y, color = 'red', lw = 2, label = 'y')
plt.legend()
plt.show()

In [ ]:
def f_prime(x: float) -> float:
    return 1 + cos(x / 10) + 3 * cos(x / 5) + 10 * cos(x / 2)

f_prime_vectorize = np.vectorize(f_prime)

fig = plt.figure(figsize=(16,9), dpi=400)
subplot = fig.add_subplot(111, facecolor = '#FFFFFF')
subplot.plot(x, f_prime_vectorize(x), color = 'red', lw = 2, label = 'y')
plt.legend()
plt.show()

In [ ]:
from numpy.random import normal

y_noise = y + normal(loc=0, scale=1, size=len(y))
fig = plt.figure(figsize=(16,9), dpi=400)
subplot = fig.add_subplot(111, facecolor = '#FFFFFF')
subplot.plot(x, y_noise, color = 'red', lw = 2, label = 'y_noise')
plt.legend()
plt.show()

In [ ]:
3 in range(10)